In [1]:
from gpu_rocket import torchrocket
import numpy as np
import torch

In [2]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)
num_examples = 1
num_channels = 20
timesteps = 100

cuda:0


In [3]:
# either: 
# -use the kernels from sktime (and then convert to 32bit if you want 32bit gpu kernels)
# -generate them yourself
# -or use the one I have edited from sktime to make 32bit 

kernels = torchrocket._generate_kernels_sktime_32bit_version(timesteps, 10_000, num_channels, seed=None)
# Note: it is required to convert each of these np arrays to lists if you want to script the model (script doesnt work with np)
# e.g. channel_indices = channel_indices.tolist() etc

In [4]:
X = np.random.rand(num_examples,num_channels,timesteps).astype(np.float32)
X_torch =torch.from_numpy(X.astype(np.float32)).to(device)

In [5]:
torch_rocket_softPPV_10k = torchrocket.TorchRocket(kernels,ppv_ver='softPPV',softppv_param=10_000.)
torch_rocket_softPPV_10k.to(device)
torch_rocket_softPPV_10k.eval()
torch_rocket_softPPV_5 = torchrocket.TorchRocket(kernels,ppv_ver='softPPV',softppv_param=5.)
torch_rocket_softPPV_5.to(device)
torch_rocket_softPPV_5.eval()
torch_rocket_heaviside_PPV = torchrocket.TorchRocket(kernels,ppv_ver='heaviside_PPV')
torch_rocket_heaviside_PPV.to(device)
torch_rocket_heaviside_PPV.eval()
print(f'')

In [6]:
with torch.no_grad():
    out_softPPV_10k = torch_rocket_softPPV_10k(X_torch)
    out_softPPV_5 = torch_rocket_softPPV_5(X_torch)
    out_heaviside_PPV = torch_rocket_heaviside_PPV(X_torch)

In [9]:
# compare to sktime implementation
from sktime.transformations.panel.rocket import _rocket
# Note: sktime rocket uses 64bit floats (we changed the sktime code to use 32bit for our timings)
(weights,lengths,biases,dilations,paddings,
        num_channel_indices,channel_indices) = kernels
weights = weights.astype(np.float64)
biases = biases.astype(np.float64)

kernels_64 = (weights,lengths,biases,dilations,paddings,
    num_channel_indices,channel_indices)

out_sktime_rocket = _rocket._apply_kernels(X.astype(np.float64), kernels_64)

In [10]:
print(f'out_softPPV shape: {out_softPPV_10k.shape}')
print(f'out_softPPV shape: {out_softPPV_5.shape}')
print(f'out_heaviside_PPV shape: {out_heaviside_PPV.shape}')
print(f'out_sktime_rocket shape: {out_sktime_rocket.shape}')

out_softPPV shape: torch.Size([20000])
out_softPPV shape: torch.Size([20000])
out_heaviside_PPV shape: torch.Size([20000])
out_sktime_rocket shape: (1, 20000)


In [11]:
for i in range(len(out_softPPV_5)):
    print(f'{i}: {out_softPPV_5[i]},{out_softPPV_10k[i]},{out_heaviside_PPV[i]},{out_sktime_rocket[0][i]}')

0: 0.2521902620792389,0.32500001788139343,0.32500001788139343,0.325
1: 5.15195369720459,5.15195369720459,5.15195369720459,5.151954016204903
2: 0.5094565153121948,0.699999988079071,0.699999988079071,0.7
3: 5.190835952758789,5.190835952758789,5.190835952758789,5.190835944436475
4: 0.3809015452861786,0.4868420958518982,0.4868420958518982,0.4868421052631579
5: 3.5817177295684814,3.5817177295684814,3.5817177295684814,3.5817179969195214
6: 0.22789111733436584,0.3913043439388275,0.3913043439388275,0.391304347826087
7: 2.3174610137939453,2.3174610137939453,2.3174610137939453,2.317461086749593
8: 0.4470290243625641,0.5555555820465088,0.5555555820465088,0.5555555555555556
9: 3.519810676574707,3.519810676574707,3.519810676574707,3.519810522684928
10: 0.3661403954029083,0.4899999797344208,0.4899999797344208,0.49
11: 4.764808177947998,4.764808177947998,4.764808177947998,4.764807877213736
12: 0.4576379060745239,0.5882353186607361,0.5882353186607361,0.5882352941176471
13: 3.342876434326172,3.34287643